In [1]:
import requests
import webbrowser
import shutil
import telebot
import import_ipynb

from IPython       import display
from bs4           import BeautifulSoup
from bs4.element   import Comment
from telebot       import types

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
with open('API_REDEMET.txt') as f:
    api_redemet = f.read()

with open('API_TELEGRAM_BOT.txt') as g:
    api_telegram = g.read()

In [ ]:
bot = telebot.TeleBot(api_telegram)

############################################# COMAND0 MUDANÇA DE PLANO #####################################

@bot.message_handler(commands=['mudancadeplano'])

def mudancadeplano(mensagem):
    texto_plano = """Legal! Fico feliz que tenha tomado a decisão de fazer um Upgrade no seu plano. Garanto que você não irá se arrepender!
Para mudar de plano, siga as instruções abaixo:

Acesse a área de compras da Hotmart através deste link: https://purchase.hotmart.com/ . Você deve fazer login com o mesmo e-mail da assinatura. A senha da Hotmart é diferente da senha da nossa plataforma de aulas.
Caso não lembre a senha da Hotmart, você pode alterá-la usando a função “ Esqueci minha senha ” na página de login da Hotmart .

Depois do login, uma página da Hotmart abrirá com todos os produtos que você já comprou utilizando esse e-mail.

Clique na assinatura da Control Academy e clique em Configurar pagamento.

Role a página até a opção “ Faça o Upgrade do seu Plano ” e clique em “ Clique aqui e conheça outros planos para a sua assinatura ” . 
Aparecerá uma tela com os planos disponíveis. Clique n o plano desejado, clique em “ Trocar para este plano ” e clique em “ Sim ”."""
    bot.send_message(mensagem.chat.id, texto_plano)

############################################# COMANDO CANCELAR #############################################

@bot.message_handler(commands=['cancelar'])

def cancelar(mensagem):
    texto_cancel = """Poxa...é uma pena saber que você precisa cancelar sua assinatura...Espero que o tempo  e o conteúdo aprendido na Control Academy sejam úteis! 
Para cancelar seu plano atual, siga as instruções abaixo:

Acesse https://purchase.hotmart.com/.
Selecione o produto.
Selecione "Gerenciar assinatura”.
Selecione “Deseja cancelar sua assinatura?”.
Confirme o cancelamento."""
    bot.send_message(mensagem.chat.id, texto_cancel )
       
############################################# COMANDO METAR ###############################################

@bot.message_handler(commands=['METAR'])

def METAR(mensagem):
    texto =bot.send_message(mensagem.chat.id,'Digite o Código ICAO do aeroporto que deseja saber as condições! Caso não saiba, clique /aqui')

    
################################## COMANDO DE TIRAR DÚVIDAS SOBRE O METAR #################################

@bot.message_handler(commands=['aqui'])
def aqui(mensagem):
    texto = """O código aeroportuário ICAO é um código composto por quatro letras que designa aeroportos em todo o mundo, 
contando com 180 países membros. 
É definido pela Organização da Aviação Civil Internacional.
    
Caso você não saiba o código ICAO do aeroporto de sua cidade, acesse este link: https://pt.wikipedia.org/wiki/Lista_de_aeroportos_do_Brasil_por_c%C3%B3digo_aeroportu%C3%A1rio_ICAO e depois apenas cole o código de quatro dígitos"""
    
    bot.send_message(mensagem.chat.id, texto)

############################################# COMANDO SIGWX #################################################

@bot.message_handler(commands=['SIGWX'])
def SIGWX(mensagem):
    link_sigwx = f'https://api-redemet.decea.mil.br/produtos/sigwx?api_key={api_redemet}'
    bot.send_message(mensagem.chat.id, 'Só um instante! Sua carta SIGWX está a caminho :)')
    requi_sigwx = requests.get(link_sigwx)
    if  requi_sigwx.status_code == 200:
        link_pic = requi_sigwx.text
        bot.send_photo(mensagem.chat.id,photo = link_pic )
    else:
        bot.send_message(mensagem.chat.id, 'Infelizmente houve um erro no servidor :(')

############################################# SATELITE IMAGES ############################################# 

@bot.message_handler(commands=['sat'])
def sat(mensagem):
    req_sat = types.ReplyKeyboardMarkup(row_width=2,one_time_keyboard=True)
    satr1 = types.KeyboardButton('North Am/South Am')
    satr2 = types.KeyboardButton('America/Africa')
    satr3 = types.KeyboardButton('Europe/Africa')
    satr4 = types.KeyboardButton('Africa/Asia')
    satr5 = types.KeyboardButton('SE Asia')
    satr6 = types.KeyboardButton('Pacífico')
    req_sat.add(satr1,satr2,satr3,satr4,satr5,satr6)
    escolha = bot.send_message(mensagem.chat.id, "Escolha uma região:", reply_markup=req_sat)
    bot.register_next_step_handler(escolha, gerador)   

def getdata(url): 
    r = requests.get(url) 
    return r.text  

def gerador(mensagem):
    if mensagem.text == 'North Am/South Am':
        s = 'a'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)
        
    elif mensagem.text == 'America/Africa':
        s = 'b1'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)
        
    elif mensagem.text == 'Europe/Africa':
        s = 'c'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)
        
    elif mensagem.text == 'Africa/Asia':
        s = 'd'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)
        
    elif mensagem.text == 'SE Asia':
        s = 'e'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)
        
    else:
        s = 'f'
        htmldata = getdata(f"https://www.aviationweather.gov/satellite/intl?region={s}") 
        soup = BeautifulSoup(htmldata, 'html.parser') 
        sat0 = soup.find_all('img')
        sat1 = str(sat0[4])
        sat2 = sat1[19:-29]
        sat_image = f'https://www.aviationweather.gov{sat2}'
        bot.send_photo(mensagem.chat.id,photo = sat_image)

############################################# VERIFICAÇÃO DE CÓDIGO ICAO ############################################# 

def verificar(mensagem):
    try:
        f = mensagem.text
        url_met = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
        url_taf = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=tafs&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
        xml_met = requests.get(url_met).content
        xml_taf = requests.get(url_taf).content
        soup_met = BeautifulSoup(xml_met, "xml")
        soup_taf = BeautifulSoup(xml_taf, "xml")
        met = soup_met.find("raw_text")
        taf = soup_taf.find("raw_text")
        tafs = str(taf)
        metar = str(met)
        taf_final = tafs.replace('<raw_text>','')
        metar_final = metar.replace('<raw_text>','')
        bot.send_message(mensagem.chat.id,metar_final[:-11])
        bot.send_message(mensagem.chat.id,taf_final[:-11])
        
    except: return True
    
############################################# MENSAGEM PADRÃO #############################################

@bot.message_handler(func=verificar)
def responder(mensagem):
    texto = """Se você deseja consultar o <b>METAR</b> de alguma localidade, clique na opção /METAR.

Para receber a <b>carta SIGWX</b> mais atualizada, clique na opção /SIGWX .

Temos também a opção de solicitar <b>imagens de satélite</b> usando o comando /sat ."""
    #print(mensagem)
    bot.reply_to(mensagem, texto, parse_mode= 'HTML')
#Olá! Seja bem-vindo(a) ao bot da Control Academy!
#Clique no item /mudancadeplano para receber um tutorial rápido de como mudar seu plano da Control Academy.
#Se deseja cancelar sua assinatura, clique na opção /cancelar que você receberá um tutorial de como proceder com o cancalamento.
    
bot.polling()

In [5]:
###################################### LABORATORY ############################################## 
# Here's where you can try and test all the requests, before running them inside the main code #
################################################################################################

bot = telebot.TeleBot(api_telegram)


@bot.message_handler(commands=['codigo'])
def codigo(mensagem):
    f = bot.send_message(mensagem.chat.id,'Digite o Código ICAO do aeroporto que deseja saber as condições! Caso não saiba, clique /aqui')
    #info_met.append[f]
    #f = mensagem.text
    #print(info_met)
    
def meteoro(mensagem):
    req_met = types.ReplyKeyboardMarkup(row_width=2,one_time_keyboard=True)
    metc1 = types.KeyboardButton('METAR')
    metc2 = types.KeyboardButton('TAF')
    metc3 = types.KeyboardButton('METAR/TAF')
    req_met.add(metc1,metc2,metc3)
    escolha = bot.send_message(mensagem.chat.id, "Escolha uma opção:", reply_markup=req_met)
    bot.register_next_step_handler(escolha, gerador_met)
    return escolha
    
def gerador_met(mensagem):
    icao = codigo(mensagem)
    escolha = meteoro(mensagem)
    if escolha == 'METAR':
        url = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={icao}"
        xml_data = requests.get(url).content
        soup = BeautifulSoup(xml_data, "xml")
        met = soup.find("raw_text")
        metar = str(met)
        metar_final = metar.replace('<raw_text>','')
        bot.send_message(mensagem.chat.id,metar_final[:-11])
    
    elif escolha == 'TAF':
        #f = 'SBBI'
        url = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=tafs&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={icao}"
        xml_data = requests.get(url).content
        soup = BeautifulSoup(xml_data, "xml")
        taf = soup.find("raw_text")
        tafs = str(taf)
        taf_final = tafs.replace('<raw_text>','')
        bot.send_message(mensagem.chat.id,taf_final[:-11])
        
    else:
        #f = 'SBBI'
        url_metar = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
        url_taf = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=tafs&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
        xml_metar = requests.get(url_metar).content
        xml_taf = requests.get(url_taf).content
        soup_metar = BeautifulSoup(xml_metar, "xml")
        soup_taf = BeautifulSoup(xml_taf, "xml")
        met = soup_metar.find("raw_text")
        taf = soup_taf.find("raw_text")
        metar = str(met)
        tafs = str(taf)
        metar_final = metar.replace('<raw_text>','')
        taf_final = tafs.replace('<raw_text>','')
        bot.send_message(mensagem.chat.id,metar_final[:-11])
        bot.send_message(mensagem.chat.id,taf_final[:-11])
        
bot.polling()

2022-04-05 17:00:31,234 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"


In [17]:
################################### LABORATORY 02 ############################################## 
# Here's where you can try and test all the requests, before running them inside the main code #
################################################################################################

bot = telebot.TeleBot(api_telegram)


def verificar(mensagem):
    f = mensagem.text
    url = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
    xml_data = requests.get(url).content
    soup = BeautifulSoup(xml_data, "xml")
    met = soup.find("raw_text")
    metar = str(met)
    metar_final = metar.replace('<raw_text>','')
    bot.send_message(mensagem.chat.id,metar_final[:-11])
        
    url = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=tafs&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
    xml_data = requests.get(url).content
    soup = BeautifulSoup(xml_data, "xml")
    taf = soup.find("raw_text")
    tafs = str(taf)
    taf_final = tafs.replace('<raw_text>','')
        
    req_taf = types.ReplyKeyboardMarkup(row_width=2,one_time_keyboard=True)
    taf1 = types.KeyboardButton('Sim')
    taf2 = types.KeyboardButton('Não')
    req_taf.add(taf1,taf2)
    escolha = bot.send_message(mensagem.chat.id, "Deseja obter também o TAF desta localidade?", reply_markup=req_taf)
    #bot.register_next_step_handler(escolha, gerador_taf())
    if escolha == 'Sim':
        print(taf_final[:-11])
        bot.send_message(mensagem.chat.id,taf_final[:-11])
    else:
        bot.send_message(mensagem.chat.id, 'Ok')

        

                
    


@bot.message_handler(func=verificar)
def responder(mensagem):
    texto = """Olá! Seja bem-vindo(a) ao bot da Control Academy!
Clique no item /mudancadeplano para receber um tutorial rápido de como mudar seu plano da Control Academy.
Se deseja cancelar sua assinatura, clique na opção /cancelar que você receberá um tutorial de como proceder com o cancalamento.

Agora, se você deseja consultar o <b>METAR</b> de alguma localidade, clique na opção /METAR.

Para receber a <b>carta SIGWX</b> mais atualizada, clique na opção /SIGWX .

Temos também a opção de solicitar <b>imagens de satélite</b> usando o comando /sat ."""
    #print(mensagem)
    bot.reply_to(mensagem, texto, parse_mode= 'HTML')


bot.polling()

2022-04-05 17:44:44,332 (__init__.py:688 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message text is empty"


In [12]:
################################### LABORATORY 03 ############################################## 
# Here's where you can try and test all the requests, before running them inside the main code #
################################################################################################


bot = telebot.TeleBot(api_telegram)



@bot.message_handler(commands=['meteorologia'])

def meteorologia(mensagem):
    f = bot.send_message(mensagem.chat.id,'Digite o Código ICAO do aeroporto que deseja saber as condições! Caso não saiba, clique /aqui')
    bot.register_next_step_handler(f)
    #f = mensagem.text
    url_metar = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
    xml_metar = requests.get(url_metar).content
    soup_metar = BeautifulSoup(xml_metar, "xml")
    met = soup_metar.find("raw_text")
    metar = str(met)
    metar_final = metar.replace('<raw_text>','')
    #bot.send_message(mensagem.chat.id,metar_final[:-11])
        
    url_taf = f"https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=tafs&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={f}"
    xml_taf = requests.get(url_taf).content
    soup_taf = BeautifulSoup(xml_metar, "xml")
    taf = soup_taf.find("raw_text")
    tafs = str(taf)
    taf_final = tafs.replace('<raw_text>','')
    
    print(met)
    print(taf)
    #bot.register_next_step_handler( mensagem , poll )
    
#def poll (mensagem):
#    req_taf = types.ReplyKeyboardMarkup(row_width=2,one_time_keyboard=True)
#    taf1 = types.KeyboardButton('Sim')
#    taf2 = types.KeyboardButton('Não')
#    req_taf.add(taf1,taf2)
#    escolha = bot.send_message(mensagem.chat.id, "Deseja obter também o TAF desta localidade?", reply_markup=req_taf)
#    bot.register_next_step_handler(escolha, gerador_taf)
        
        
#def gerador_taf(mensagem):
#    if mensagem.text == 'Sim':
#        bot.send_message(mensagem.chat.id,infos['taf_final'[:-11]])
#        bot.send_message(mensagem.chat.id,infos['metar_final'[:-11]])
#    else:
#        bot.send_message(mensagem.chat.id,infos[metar_final[:-11]])
        
def verificar(mensagem):
    return True
    
@bot.message_handler(func=verificar)
def responder(mensagem):
    texto = """Se você deseja consultar o <b>METAR</b> de alguma localidade, clique na opção /meteorologia.

Para receber a <b>carta SIGWX</b> mais atualizada, clique na opção /SIGWX .

Temos também a opção de solicitar <b>imagens de satélite</b> usando o comando /sat ."""
    #print(mensagem)
    bot.reply_to(mensagem, texto, parse_mode= 'HTML')    
    
    
bot.polling()

TypeError: register_next_step_handler() missing 1 required positional argument: 'callback'